# Convert data files to .json

### Load data

In [3]:
import os
import json
import numpy as np
import pandas as pd
from data import plot_data, nighttime_frac
from load_fims_spear_maps import integrated_h2_map

plot_data = plot_data.drop('Hover Text', axis=1)

### Prepare plot_data

In [4]:
### fix sizes 

mags = np.array(plot_data['Apparent Magnitude'])

pt1 = [np.min(mags.astype('float')), 30]
pt2 = [np.max(mags.astype('float')), 3]
tau = (pt2[0]-pt1[0]) / np.log(pt1[1]/pt2[1])
A = pt1[1] * np.exp(pt1[0] / tau)
sizes = A * np.exp(-mags.astype('float') / tau)

plot_data["Size"] = sizes

### add hasspectra column

hasspectra = np.zeros(plot_data.shape[0]).astype('bool')
iue_paths = np.zeros(plot_data.shape[0]).astype('U64')
names = os.listdir('../v2-static-ob-star-visualizer/static/data/spectra')
for i, name in enumerate(plot_data["Name"]+".json"):

    if name.replace(' ', '_') in names:
        hasspectra[i] = True
        iue_paths[i] = 'static/data/spectra/' + name.replace(' ', '_')

plot_data["HasSpectra"] = hasspectra
plot_data["IUESpectra"] = iue_paths

### Prepare integrated_h2_map

In [5]:
latitudes = np.linspace(-90, 90, integrated_h2_map.shape[0]).tolist()
longitudes = np.linspace(0, 360, integrated_h2_map.shape[1]).tolist()
z_values = integrated_h2_map.tolist()

integrated_h2_map = {
    "x": longitudes,
    "y": latitudes,
    "z": z_values
}

### Export .json files

In [6]:
plot_data.to_json('../v2-static-ob-star-visualizer/static/data/plot_data.json', orient='records')

with open('../v2-static-ob-star-visualizer/static/data/integrated_h2_map.json', 'w') as f:
    json.dump(integrated_h2_map, f)

# pd.DataFrame(nighttime_frac).to_json('../v2-static-ob-star-visualizer/static/data/nighttime_frac.json', orient='records')

# for file_name in os.listdir('../../ob_catalogue/ob_catalogue_spectra/'):
#     if file_name[-4:] == '.csv':
#         spectrum = pd.DataFrame(np.genfromtxt('../../ob_catalogue/ob_catalogue_spectra/' + file_name, delimiter=',', dtype='float'))
#         plot_data.to_json(f'../static-ob-star-visualizer/static/data/spectra/{file_name[:-4]}.json', orient='records')

### Prepare and export OB star spectra as .json files

In [12]:
for name in os.listdir('../../ob_catalogue/ob_catalogue_spectra/'):
    if name.endswith('.csv'):
        arr = np.loadtxt('../../ob_catalogue/ob_catalogue_spectra/' + name, delimiter=',', dtype='float')

        if arr.ndim == 2:
            wavelengths = arr[:,:int(arr.shape[1]/2)].tolist()
            fluxes = arr[:,int(arr.shape[1]/2):].tolist()
        else:
            wavelengths = arr[:int(arr.shape[0]/2)].tolist()
            fluxes = arr[int(arr.shape[0]/2):].tolist()

        data = {
            'wavelength': wavelengths,
            'flux': fluxes
        }

        with open('../v2-static-ob-star-visualizer/static/data/spectra/' + name[:-4] + '.json', 'w') as f:
            json.dump(data, f)